In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 데이터 로딩
train_path = '/Users/songsanghun/Downloads/Mobility/FinalProject/Toa_sort.csv'
test_path = '/Users/songsanghun/Downloads/Mobility/FinalProject/Toa_test_sort.csv'

# 데이터 로딩 (헤더 없음)
columns = ['re_x', 're_y', 'anchor_id', 'TOA'] + [f'CSI_{i}' for i in range(1, 129)]
df_train = pd.read_csv(train_path, header=None, names=columns)
df_test = pd.read_csv(test_path, header=None, names=columns)

# 출력: re_x, re_y
output_cols = ['re_x', 're_y']

# 입력 특징: CSI_1 to CSI_128 (magnitude) + anchor_id (one-hot encoded)
csi_cols = [f'CSI_{i}' for i in range(1, 129)]

# anchor_id one-hot encoding
encoder = OneHotEncoder(sparse=False)
anchor_train_encoded = encoder.fit_transform(df_train[['anchor_id']])
anchor_test_encoded = encoder.transform(df_test[['anchor_id']])

# 입력 데이터 결합
X_train = np.concatenate([df_train[csi_cols].values, anchor_train_encoded], axis=1)
y_train = df_train[output_cols].values

X_test = np.concatenate([df_test[csi_cols].values, anchor_test_encoded], axis=1)
y_test = df_test[output_cols].values

# 정규화
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# 입력 차원 계산
input_dim = X_train.shape[1]

# DNN 모델 구축
model = Sequential([
    Dense(128, activation='relu', input_shape=(input_dim,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(2)  # 출력: x, y
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 모델 학습
history = model.fit(X_train_scaled, y_train_scaled, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# 평가
loss, mae = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
print(f'Test Loss: {loss:.4f}, Test MAE: {mae:.4f}')

# 예측 예시
predictions = model.predict(X_test_scaled)
predictions_original = scaler_y.inverse_transform(predictions)
actual_original = scaler_y.inverse_transform(y_test_scaled)

# RMSE 계산
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(actual_original, predictions_original))
print(f'RMSE: {rmse:.4f}')

# 모델 저장
model.save('dnn_position_model.h5')
print("모델이 'dnn_position_model.h5'로 저장되었습니다.")

: 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.scatter(actual_original[:, 0], actual_original[:, 1], label='Actual', alpha=0.5)
plt.scatter(predictions_original[:, 0], predictions_original[:, 1], label='Predicted', alpha=0.5)
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Actual vs Predicted Positions')
plt.legend()
plt.show()